# MGGDATOS - INFORME - Contratos desde 2025

In [1]:
from IPython.display import Markdown as md
import pandas as pd

In [2]:
contratos = pd.read_csv("./imports/contratos_desde_01_01_2021.csv")

In [3]:
total_contratos = len(contratos)

In [4]:
contratos.sample(1)

,objeto_contrato,entidad_adjudicadora,procedimiento_contratacion,tramitacion_tipo,lote,adjudicacion_fecha,adjudicataria_nif,importe_ofertado,expediente_numero,licitacion_enlace,...,publicacion_fecha,publicacion_medio,licitacion_resultado,pliego_enlace,perfil_contratante_enlace,financiacion_europea,fuente_financiacion_id,fuente_financiacion,localizador_programa_ue,formula_revision_precios
64069,Atomizador hidroneumático suspendido 200L ATAS...,Presidencia del Instituto canario de Investiga...,Contrato menor,Ordinaria,_Z,2024/06/17 00:00:00.000,B35901834,4862,CONTM/2024/0000032709,https://contrataciondelestado.es/wps/poc?uri=d...,...,2024/10/02 00:00:00.000000000,Perfil del contratante,Adjudicado,_U,https://contrataciondelestado.es/sindicacion/C...,NO,_U,_U,_U,_U


## Limpieza y transformación de datos iniciales

Primero nos quedamos con las columnas que nos interesan:

In [5]:
contratos = contratos[[
    'objeto_contrato',
    'entidad_adjudicadora',
    'adjudicacion_fecha',
    'adjudicataria_nif',
    'licitacion_estado',
    'clasificacion_cpv',
    'contrato_tipo',
    'publicacion_fecha',
    'licitacion_resultado',
]]
    

Y ahora renombramos las columnas para que sean un poco más cortas:

In [6]:
contratos = contratos.rename(columns={
    'objeto_contrato': 'objeto',
    'entidad_adjudicadora': 'adjudicador',
    'adjudicataria_nif': 'nif_adjudicatario',
    'licitacion_estado': 'estado',
    'clasificacion_cpv': 'cpv',
    'contrato_tipo': 'tipo',
    }, inplace=False)
contratos.sample(2)

,objeto,adjudicador,adjudicacion_fecha,nif_adjudicatario,estado,cpv,tipo,publicacion_fecha,licitacion_resultado
37035,Suministro de equipos de climatización para di...,Dirección General de Telecomunicaciones y Nuev...,2021/12/21 00:00:00.000,42864613L,Resuelta,42510000,Suministros,2021/12/21 00:00:00.000000000,Adjudicado
77531,"Reparación de cancha grande del IES Lilas, T.M...",Dirección General de Infraestructuras y Equipa...,2024/12/20 00:00:00.000,B76338243,Resuelta,_U,Obras,2025/02/04 00:00:00.000000000,Adjudicado


## Filtrado

Actualmente hay **{eval}`total_contratos`** contratos.

### filtro por resultado de la licitación

¿Qué tipos de resultados hay para un contrato?

In [7]:
contratos[['licitacion_resultado', 'objeto']].groupby('licitacion_resultado').count()

,objeto
licitacion_resultado,
Adjudicado,184294
Desierto,4114
Desistimiento,1868
Formalizado,12826
Renuncia,103


Podemos eliminar los de tipo `Desierto`, `Desistimiento` y `Renuncia`:

In [8]:
desiertos = contratos['licitacion_resultado'] == 'Desierto'
num_desiertos = sum(desiertos)

desistimientos = contratos['licitacion_resultado'] == 'Desistimiento'
num_desistimientos = sum(desistimientos)

renuncias = contratos['licitacion_resultado'] == 'Renuncia'
num_renuncias = sum(renuncias)

contratos = contratos[~(desiertos | desistimientos | renuncias)]
num_contratos = len(contratos)
contratos[['licitacion_resultado', 'objeto']].groupby('licitacion_resultado').count()

,objeto
licitacion_resultado,
Adjudicado,184294
Formalizado,12826


Nos quedamos con un total de **{eval}`num_contratos`** contratos.

### Filtramos por tipo de contrato

¿Cuáles son los tipos de contrato?

In [9]:
contratos[['tipo', 'objeto']].groupby('tipo').count()

,objeto
tipo,
Administrativo especial,103
Colaboración entre el sector público y sector privado,4
Obras,13361
Patrimonial,44
Privado,641
Servicios,85609
Suministros,97358


¿Podriamos descartar los de tipo `Obras`, `Patrimonial`, `Suministros`, etc? Nos
quedaríamos solo con los de tipo `Privado` y tipo `Servicios`. Esto nos
debería reducir el número de contratos a {eval}`sum(contratos['tipo'] == 'Servicios')` +
{eval}`sum(contratos['tipo'] == 'Privado')`.

In [10]:
servicios = contratos['tipo'] == 'Servicios'
num_servicios = sum(servicios)
privados = contratos['tipo'] == 'Privado'
num_privados = sum(privados)
contratos = contratos[servicios | privados]
num_contratos = len(contratos)

Nos quedamos con un total de {eval}`num_servicios` + {eval}`num_privados` =  **{eval}`num_contratos`** contratos.

### Filtramos por estado de contrato

¿En qué estado puede estar un contrato?

In [11]:
contratos[['estado', 'objeto']].groupby('estado').count()

,objeto
estado,
Adjudicada,2031
Anulada,307
Resuelta,83912


¿Cual es la diferencia entre `Adjudicada` y `Resuelta`? ¿_qui lo sa_? Pero
parece que quedarnos solo con las adjudicadas:

In [12]:
adjudicados = contratos['estado'] == 'Adjudicada'
contratos = contratos[adjudicados]
num_contratos = len(contratos)

Nos quedamos con un total de **{eval}`num_contratos`** contratos.

Los campos `publicacion_fecha` y `adjudicacion_fecha` son cadenas de texto:

In [13]:
contratos['publicacion_fecha'].describe()

count                              2031
unique                              393
top       2021/01/22 00:00:00.000000000
freq                                116
Name: publicacion_fecha, dtype: object

In [14]:
contratos['adjudicacion_fecha'].describe()

count                        2031
unique                        442
top       2020/07/01 00:00:00.000
freq                          192
Name: adjudicacion_fecha, dtype: object

Hay que convertirlos a campos de tipo fecha (`datetime.date`)

In [15]:
contratos['f_publicacion'] = pd.to_datetime(contratos['publicacion_fecha'])
contratos['f_adjudicacion'] = pd.to_datetime(contratos['adjudicacion_fecha'])
contratos.sample(2)

,objeto,adjudicador,adjudicacion_fecha,nif_adjudicatario,estado,cpv,tipo,publicacion_fecha,licitacion_resultado,f_publicacion,f_adjudicacion
9550,Servicio de mantenimiento y conservación de lo...,Viceconsejería de Políticas Sociales y Vivienda,2020/02/20 00:00:00.000,A38363917,Adjudicada,35113300,Servicios,2019/07/25 00:00:00.000000000,Adjudicado,2019-07-25,2020-02-20
19012,El objeto del contrato será la prestación de l...,Consejero Delegado del Instituto Canario de De...,2021/06/25 00:00:00.000,B96081526,Adjudicada,92000000,Servicios,2021/09/14 00:00:00.000000000,Adjudicado,2021-09-14,2021-06-25


Solo nos interesan contratos publicados desde el año pasado, así que los filtramos:

In [16]:
contratos = contratos[contratos.f_publicacion.dt.year >= 2025]
num_contratos = len(contratos)

Nos quedamos con un total de **{eval}`num_contratos`** contratos.

In [17]:
contratos.sort_values(by=['nif_adjudicatario', 'adjudicador']).head(3)


,objeto,adjudicador,adjudicacion_fecha,nif_adjudicatario,estado,cpv,tipo,publicacion_fecha,licitacion_resultado,f_publicacion,f_adjudicacion
59394,Dirección de Ejecución para la obra de evacuac...,Dirección General de Recursos Económicos del S...,2025/03/06 00:00:00.000,00814958E,Adjudicada,71200000,Servicios,2025/03/07 00:00:00.000000000,Adjudicado,2025-03-07,2025-03-06
100967,Dirección de ejecución para la obra e la ud de...,Dirección General de Recursos Económicos del S...,2025/08/11 00:00:00.000,00814958E,Adjudicada,71200000,Servicios,2025/08/11 00:00:00.000000000,Adjudicado,2025-08-11,2025-08-11
106053,Dirección de ejecución para la obra de Instala...,Dirección General de Recursos Económicos del S...,2025/06/13 00:00:00.000,00814958E,Adjudicada,71200000,Servicios,2025/06/13 00:00:00.000000000,Adjudicado,2025-06-13,2025-06-13


In [18]:
pd.set_option('display.max_colwidth', None)
adjudicadores = contratos[['objeto', 'adjudicador']].sort_values(by='adjudicador').groupby(by='adjudicador').count()

In [19]:
buff = []
for organo, num_contratos in adjudicadores.itertuples():
    buff.append(f'## {organo} ({num_contratos} contratos)\n')
    subrows = contratos[contratos['adjudicador'] == organo]
    for subrow in subrows.itertuples():
        buff.append(f'- {subrow.objeto}\n')
    buff.append('\n')
md('\n'.join(buff))

## Cartográfica de Canarias S.A. (1 contratos)

- Metaanálisis sobre la coexistencia de usos y actividades en el medio marino de la provincia occidental de la Demarcación Marina Canaria.



## Consejera Delegada de Asistencia Integral Tributaria, S.A (Asista Canarias) (1 contratos)

- Servicios de diseño, desarrollo, implantación y mantenimiento del sistema de información de contabilidad de los tributos. 



## Consejero Delegado de la Sociedad Canaria de Fomento Económico S.A. - PROEXCA (1 contratos)

- Servicio de asistencia técnica a la oficina de proyectos estratégicos creada en Proexca para el apoyo en la gestión de los proyectos estratégicos transformadores para la economía de Canarias.



## Consejería Delegada de Gestión y Planeamiento Territorial y Medioambiental, S.A. (1 contratos)

- Servicio para la elaboración del Plan Director de Seguridad de la Información en Gesplan



## Consejería Delegada de Hoteles Escuela de Canarias, S.A. (3 contratos)

- Servicios de asesoramiento y consulta jurídica en materia laboral, así como la dirección, asistencia, representación y defensa en los procesos cuyo
conocimiento se atribuye el orden jurisdiccional social, para la sociedad mercantil pública Hoteles Escuela de Canarias, S.A.

- Servicios consistentes en el diseño, organización e impartición de cursos de capacitación inicial en distintas profesiones del ámbito de la hostelería y el turismo, dirigidos a personas vulnerables para su inserción laboral, para HECANSA con financiación “Next Generation EU”, con cargo al mecanismo de recuperación y resiliencia (MRR) 



- Servicios de asesoramiento y consulta jurídica en materia laboral, así como la dirección, asistencia, representación y defensa en los procesos cuyo
conocimiento se atribuye el orden jurisdiccional social, para la sociedad mercantil pública Hoteles Escuela de Canarias, S.A.



## Consejería de Bienestar Social, Igualdad, Juventud, Infancia y Familias (1 contratos)

- Servicios postales de notificación de actos administrativos en materia de reconocimiento de la Discapacidad



## Consejería de Educación, Formación Profesional, Actividad Física y Deportes de Canarias (3 contratos)

- Servicio de organización y realización de estancias formativas (formación, traslado, alojamiento y póliza de seguro) de cuatro semanas de duración, en Irlanda, Francia, Alemania e Italia, en el mes de julio de 2025, con programas de formación e inmersión lingüística, en un contexto intercultural y de inmersión lingüística.

- Servicios de desarrollo e implantación del nuevo sistema de información para la gestión educativa de la Consejería de Educación, Formación Profesional, Actividad Física y Deportes del Gobierno de Canarias.

- Servicio de organización y realización de estancias formativas (formación, traslado, alojamiento y póliza de seguro) de cuatro semanas de duración, en Irlanda, Francia, Alemania e Italia, en el mes de julio de 2025, con programas de formación e inmersión lingüística, en un contexto intercultural y de inmersión lingüística.



## Consejería de Sanidad del Gobierno de Canarias (1 contratos)

- servicio de mantenimiento integral del edificio, instalaciones y elementos fijos de equipamiento del Hospital Universitario de Gran Canaria Dr. Negrín



## Dirección General de Agricultura de la Consejería de Agricultura Ganadería, Pesca y Soberanía Alimentaria de la Comunidad Autónoma de Canarias (1 contratos)

- Servicio de propagación por cultivo in vitro de un mínimo de 150.000 plantas de piña tropical de Roja Española debidamente endurecidas



## Dirección General de Aguas de la Consejería de Política Territorial, Cohesión Territorial y Aguas del Gobierno de Canarias (1 contratos)

- Servicio de redacción para la elaboración del Proyecto constructivo: Colectores de saneamiento del casco urbano de El Pueblo yLodero, Villa de Mazo, conexión con la E.D.A.R. comarcal (La Palma).



## Dirección General de Dependencia (2 contratos)

- Servicio de ayuda a domicilio dirigido a las personas en situación de dependencia y servicio de supervisión y control de calidad de los servicios domiciliarios, en las islas de Fuerteventura, Lanzarote, La Gomera y Tenerife, tramitado por la Consejería de Bienestar Social, Igualdad, Juventud, Infancia y Familias y promovido por la Dirección General de Dependencia.

- Servicios postales de notificación actos administrativos en materia de reconocimiento de Dependencia



## Dirección General de Estudios, Formación e Investigación en Seguridad Pública de la Consejería de Presidencia, Administraciones Públicas, Justicia y Seguridad del Gobierno de Canarias (2 contratos)

- El objeto del contrato será la realización del "Servicio de impartición de formación en conducción evasiva para diversos cursoscon el fin de garantizar la impartición de los cursos y formación necesaria en dicha materia, para atender las necesidades deformación y funcionamiento responsabilidad de esta Dirección General.

- contratación del servicio de limpieza de las dependencias de la Dirección General en en la isla Tenerife, mediante procedimientoabierto y tramitación ordinaria. SER/2025/0000047947



## Dirección General de Formación Profesional y Enseñanzas de Régimen Especial de la Comunidad Autónoma de Canarias (1 contratos)

- Realización del servicio de gestión de inclusión en el sistema de la Seguridad Social y de tramitación del pago a la Tesorería dela Seguridad Social de las cotizaciones sociales del alumnado de formación profesional y enseñanzas de régimen especial que realiza prácticas formativas en empresas u organismos equiparados.



## Dirección General de Infraestructuras y Equipamientos del Gobierno de Canarias (6 contratos)

- Coordinación de la Seguridad y Salud en fase de ejecución de obra (CSS) para la obra Acondicionamiento del acceso principal en elCEIP Valverde, Valverde, el Hierro.

- La coordinación en materia de seguridad y salud en fase de ejecución de obra de la Obra Construccion de un Aula Enclave con bañoadaptado CEIP El Fraile, Arona, con el fin de dar seguimiento, vigilar y controlar el cumplimiento de las medidas de seguridadrecogidas en el plan de seguridad y salud.

- La dirección de obra (DO)de la obra de Construcción de un Aula Enclave con baño adaptado CEIP El Fraile, Arona, con el fin dedirigir el desarrollo de la obra, aspectos técnicos, estéticos, urbanísticos y medioambientales.

- Dirección de Obra (DO) del proyecto denominado "Acondicionamiento del acceso principal en el CEIP Valverde, Valverde, el Hierro"

- La dirección de ejecución de obra (DEO) Obra de construcción de un Aula Enclave con Baño Adaptado CEIP El Fraile, Arona, con elfin de dirigir la ejecución material de la obra y controlar la calidad de lo edificado.

- Dirección de Ejecución de Obra (DEO) para la obra Acondicionamiento del acceso principal en el CEIP Valverde, Valverde, el Hierro.



## Dirección General de Mayores y Participación Activa (1 contratos)

- la realización del Servicio de Viajes de Turismo Social para el colectivo de  personas mayores  de 55 años, residentes enCanarias,  y acompañante los cónyuges, parejas y familiares (CANARIAS SE MUEVE) residentes en la Comunidad Autónoma de Canarias



## Dirección General de Patrimonio Cultural del Gobierno de Canarias (3 contratos)

- I Simposio: Historia, memoria e identidad.

- Restauración de procesos de las Real Audicencia de Canarias depositados en el Archivo Histórico Provincial de Las Palmas

- Restauración de documentos del Archivo Histórico Provincial de Las Palmas 



## Dirección General de Patrimonio y Contratación (1 contratos)

- El objeto de la contratación es el servicio de inspección técnica reglamentaria de los referidos ascensoresde los ESM de LPGC y deTenerife.



## Dirección General de Promoción y Diversificación Económica del Gobierno de Canarias (2 contratos)

- Campaña de comunicación del proyecto "Educación Financiera"

- La organización del evento Reunión Anual Nacional de la EEN en Tenerife



## Dirección General de Recursos Económicos del Servicio Canario de la Salud (35 contratos)

- Coordinador de Seguridad y salud para la obra de reforma de instalaciones del servicio de anatomía patológica del  hospital Dr. José Molina Orosa de Lanzarote

- Servicio de Redacción de Proyecto de  terminación del Centro de Salud de Valverde en el Hierro

- Dirección de obra de instalaciones para la obra de Reforma de Instalaciones del Servicio de Anatomía patológica del Hospital Dr. José Molina Orosa de Lanzarote

- Dirección de Ejecución para la obra de evacuación desde la unidad de corta estancia de la planta  5ª

- Servicio Coordinación Segur.Salud H.G Fuertevent

- Servicio de Redacción de Proyecto del Centro de Salud de el Pinar, El Hierro

- DD.FF Arquitectura Reforma Eléctrica H.U Lanzart

- Servicio de Redacción de Proyecto de ampliación y reforma zona de Urgencias del Centro de Salud de Arucas, en Gran Canaria

- Dirección de Ejecución para la obra de Legalización de deposito de combustible y caldera de vapor en el edificio industrial del HUIGC

- Contrato Servicio DD.FF de Ejecución H.G Fuert

- Levantamiento topográfico de la parcela SA-1 en la urbanización La Lajita 2000, Pájara, Fuerteventura

- DD.FF Arquitectura Obra nave H.G Fuerteventura

- Dirección de ejecución, arquitecto técnico, para la obra de reforma del Centro de Salud de San Bartolomé, Isla de Lanzarote

- Dirección de obra de ingeniería para la obra de Instalaciones de Climatización del Centro de Salud de Alcaravaneras. Gran Canaria

- Coordinador de Seguridad y salud para la Obra de evacuación desde la unidad de corta estancia

- Estudio topográfico de la parcela para la ampliación del Centro de Salud de Tafira

- Dirección de obra de instalaciones para la obra de ud de corta estancia en el CHUIMI

- Estudio topográfico de la parcela del antiguo Colegio Universitario de Las Palmas (CULP)

- Dirección de Obra de Arquitectura  para la obra de evacuación desde la unidad de corta estancia de la planta  5ª

- Dirección de Obra de Instalaciones para la obra de legalización de depósito de combustible y caldera de vapor en el HUIGC

- Dirección de ejecución para la obra e la ud de corta estancia en el CHUIMI

- Servicio de Redacción de Proyecto de  básico y de ejecución de la obra  de reforma interior para el módulo del Hospital de día en Hospital Nuestra Señora de Guadalupe, San Sebastián de la Gomera

- Coordinación de Seguridad y Salud para la obra de legalización de depósito de combustible y caldera de vapor en el edificio industrial HUIGC

- DD.FF Ingeniería Red Eléctrica H.U Lanzarote

- Dirección de ejecución para la obra de Instalaciones de Climatización del Centro de Salud de Alcaravaneras. Gran Canaria

- Servicio de Dirección de Coordinación de Seguridad y Salud para la obra de Instalaciones de Climatización del Centro de Salud de Alcaravaneras. Gran Canaria

- DD.FF Ejecución Ref.Eléctrica H.U Lanzarote

- Dirección de obra de Ingeniería para la obra de Reforma del centro de Salud de San Bartolomé, Isla de Lanzarote

- Coordinador de Seguridad y Salud para la obra de reforma del Centro de Salud de san Bartolomé, Isla de Lanzarote

- Dirección de ejecución para la obra de Reforma de Instalaciones del Servicio de Anatomía patológica del Hospital Dr. José Molina Orosa de Lanzarote

- Dirección de obra de arquitectura para la obra de la nueve ud de corta estancia en el CHUIMI

- DD.FF Coordinación Seguridad y Salud Lanzarote

- Coordinador de Seguridad y Salud para la obra de la ud de corta estancia en el CHUIMI

- Dirección Facultativa de Ingeniería Nave H.G Fuerteventura

- Dirección de obra de Arquitectura para la obra de reforma del Centro de Salud de San Bartolomé, Isla de Lanzarote



## Dirección General de Relaciones con África del Gobierno de Canarias (3 contratos)

- Servicio consistente en la consultoría y asistencia técnica para la implementación de tres actividades necesarias para la ejecución del PROYECTO ESTRATÉGICO DE APOYO A LA GOBERNANZA DE LA COOPERACIÓN INTERREG MAC 2021-2027, con acrónimo ATLANTE, que forma parte del Programa de Cooperación INTERREG VI-D MADEIRA-AZORES-CANARIAS (MAC) 2021-2027, cofinanciado por el Fondo FEDER

- Servicio consistente en la consultoría y asistencia técnica para la implementación de tres actividades necesarias para la ejecución del PROYECTO ESTRATÉGICO DE APOYO A LA GOBERNANZA DE LA COOPERACIÓN INTERREG MAC 2021-2027, con acrónimo ATLANTE, que forma parte del Programa de Cooperación INTERREG VI-D MADEIRA-AZORES-CANARIAS (MAC) 2021-2027, cofinanciado por el Fondo FEDER

- Servicio consistente en la consultoría y asistencia técnica para la implementación de tres actividades necesarias para la ejecución del PROYECTO ESTRATÉGICO DE APOYO A LA GOBERNANZA DE LA COOPERACIÓN INTERREG MAC 2021-2027, con acrónimo ATLANTE, que forma parte del Programa de Cooperación INTERREG VI-D MADEIRA-AZORES-CANARIAS (MAC) 2021-2027, cofinanciado por el Fondo FEDER



## Dirección General de Salud Pública del Gobierno de Canarias (1 contratos)

-  Servicio de mantenimiento preventivo y correctivo de los sistemas de alimentación ininterrumpida (UPS- SAI) instalados en los edificios de la Dirección General de Salud Pública del Servicio Canario de la Salud



## Dirección General de Seguridad del Gobierno de Canarias (2 contratos)

- Servicio de transformación técnica de 20 vehículos, propiedad de la Dirección General de Seguridad propietaria de vehículospoliciales y suminisro de componentes

- El objeto del contrato será la realización del servicio de mantenimiento y desarrollo evolutivo del sistema SIGESCA. El serviciodeberá cubrir el mantenimiento preventivo,correctivo, adaptativo y evolutivo, así como el soporte a las incidencias que puedansurgir durante el periodo previsto por este  contrato.



## Dirección General de Telecomunicaciones y Nuevas Tecnologías (1 contratos)

- Servicio de soporte de Bases de datos PostgreSQL corporativas del Gobierno de Canarias



## Dirección General de Transformación Digital de los Servicios Públicos (2 contratos)

- Servicio de accesibilidad versión audio de páginas Web “Readspeaker” de la Administración Pública de la Comunidad Autónoma de Canarias.

- Renovar el mantenimiento de las licencias del gestor de bases de datos Oracle 2026



## Dirección General de Transición Ecológica y Lucha Contra el Cambio Climático del Gobierno de Canarias (1 contratos)

- Prestación del servicio de asistencia técnica de direcciónfacultativa para las obras de emergencia de retirada masiva de residuoslocalizados en el cauce del Barranco y corrección del sellado del antiguo vertedero de Fernando Porto, témino municipal de Garafíaen la isla de La Palma.



## Dirección Gerencia Hospital Universitario de Gran Canaria Doctor Negrín (2 contratos)

- Contrato Basado en el Acuerdo Marco para el Servicio de Gestión Integral y de Eliminación de Residuos Sanitarios y otros Residuos de los Centros Sanitarios del Servicio Canario de la Salud.

- El objeto del presente contrato consiste en la prestación del Servicio de vigilancia, seguridad y protección, en la Unidad deSalud Mental ubicada dentro del recinto CS COSTA AYALA, centrodependiente de la Dirección Gerencia del Hospital Universitario de Gran Canaria Dr. Negrín.



## Dirección Gerencia del Complejo Hospitalario Universitario Insular Materno Infantil (2 contratos)

- Servicio de gestión integral y de eliminación de residuos sanitarios y otros residuos de los Centros Sanitarios del Servicio Canario de la Salud, mediante la suscripción de acuerdos marcos

- Servicio de derecho al uso de las licencias de software de Backup para el Complejo Hospitalario Universitario Insular-Materno Infantil de Gran Canaria.



## Dirección Gerencia del Complejo Hospitalario Universitario de Canarias (19 contratos)

- Servicio de gestión integral y de eliminación de residuos sanitarios y otros residuos de los centros sanitarios

- Servicio de cualificación de las cabinas de flujo laminar, de seguridad biológica y extractoras del Complejo HospitalarioUniversitario de Canarias (CHUC)

- Dirección de obra y coordinación de seguridad y salud de la obra civil e instalaciones eléctricas, hidrosanitarias y protección contra incendios del Centro de Atención Especializada (CAE) de La Orotava, perteneciente al Complejo Hospitalario Universitario de Canarias derivado de Sistema Dinámico de Adquisición.

- Mantenimiento Integral del Sistema de Purificación de Agua de la Unidad de Diagnóstico Molecular del Laboratorio Central delComplejo Hospitalario Universitario de Canarias

- Redacción del proyecto de las instalaciones de protección contra incendios (PCI), del Edificio Pabellón de Gobierno y del Edificio CAE San Benito, del Complejo Hospitalario Universitario de Canarias

- Coordinación de seguridad y salud del proyecto de nuevo laboratorio citostáticos Hospital del Norte, del Complejo Hospitalario Universitario de Canarias

- Dirección de obra del proyecto de nuevo laboratorio citostáticos Hospital del Norte, del Complejo Hospitalario Universitario de Canarias

- Redacción del proyecto de obra para la adecuación de espacios destinados a las instalaciones de oxígeno líquido y nitrógeno líquido del Complejo Hospitalario Universitario de Canarias

- Servicios de mantenimiento de la electrónica de red del Hospital del Norte.

- Coordinación de seguridad y salud durante la ejecución de la obra de adecuación de parámetros interiores en las habitaciones de la planta 2, del Hospital del Norte, perteneciente al CHUC

- Servicio de validación de la calidad del aire en los sistemas de filtración de Quirófanos, Paritorios, Laboratorio FIV, Laboratorio Anatomía Patológica y habitaciones de aislamientos del Complejo Hospitalario Universitario de Canarias.

- Servicio de la Dirección Facultativa de la obra civil e instalaciones del nuevo laboratorio del Hospital del Norte del Complejo Hospitalario Universitario de Canarias, correspondiente a la correspondiente a la categoría 1 “Dirección de Obras” y a la categoría 3 “Coordinación de seguridad y salud”, derivado de sistema dinámico de adquisición, mediante tramitación anticipada.

- Servicio de la Dirección Facultativa de la obra civil e instalaciones del nuevo laboratorio del Hospital del Norte del Complejo Hospitalario Universitario de Canarias, correspondiente a la correspondiente a la categoría 1 “Dirección de Obras” y a la categoría 3 “Coordinación de seguridad y salud”, derivado de sistema dinámico de adquisición, mediante tramitación anticipada.

- Redacción del proyecto técnico completo planta 1 Edif. B, destino laboratorio microbiología, sala aféresis y despachos

- Coordinación seguridad y salud del suministro con fabricación, instalación y puesta en marcha de cuatro transformadores y cuadro eléctrico intermedio de distribución Edif Hospitalizacion y NEA

- Redacción del proyecto de sustitución de enfriadora y unidad de tratamiento de aire (UTA) del Hospital del Norte

- Dirección de obra y coordinación de seguridad y salud de la obra civil e instalaciones eléctricas, hidrosanitarias y protección contra incendios del Centro de Atención Especializada (CAE) de La Orotava, perteneciente al Complejo Hospitalario Universitario de Canarias derivado de Sistema Dinámico de Adquisición.

- Contrato de Mantenimiento integral de los equipos de mamografía y esterotaxia de la marca HOLOGIC del Complejo HospitalarioUniversitario de Canarias

- Redacción del proyecto de ejecución (obra civil e instalaciones), de las obras de las salas de urgencias de pediatría y unidad de corta estancia 1, del Servicio de Urgencias



## Dirección Gerente del Hospital Universitario Nuestra Señora de Candelaria (3 contratos)

- Contrato basado del expediente de contratación del servicio de gestión integral y de eliminación de residuos sanitarios y otros residuos de los centros sanitarios del servicio canario de salud, en concreto en el lote 4.1 gerencia de la candelaria, derivado del acuerdo marco 23/S/22/SS/DG/A/AM12, mediante procedimiento abierto de adjudicación y tramitación ordinaria.

- Servicio de mantenimiento y soporte para el sistema INTELLISPACE ISCV, para el Servicio de Cardiología del Complejo HospitalarioUniversitario Nuestra Señora de Candelaria a realizar por la empresa Philips Ibérica S.A.,   mediante un procedimiento negociadosin publicidad sin concurrencia de ofertas y tramitación anticipada

- Servicio de recogida, transporte y distribución de ropa hospitalaria entre lavanderia y el Hospital del Suir del CHUNSC,medianteprocedimiento abierto simplificado sumario y tramitación anticipada.



## Dirección de la Agencia Tributaria Canaria (2 contratos)

- El objeto de este contrato es la prestación de los servicios para la limpieza integral de las dependencias administrativas de laATC ubicadas en el Edificio Royal

- Servicio de asistencia técnica para la administración y evolución de bases de datos Oracle en la Agencia Tributaria Canaria.



## Dirección del Consorcio para el Diseño, la Construcción, el Equipamiento y la Explotación de la Plataforma Oceánica de Canarias (1 contratos)

- Servicio de embarcación para la realización de trabajos marítimos, suministros y transporte de personal y material/equipamiento para el Consorcio para el diseño, construcción, equipamiento y explotación de la Plataforma Oceánica de Canarias.



## Dirección del Servicio Canario de Salud (1 contratos)

- El presente servicio tiene como objetivo el desarrollo, implantación y mantenimiento de una solución tecnológica integral para la gestión de proyectos corporativos, que proporcione una visión estructurada, trazable y escalable del ciclo de vida completo de los proyectos, alineada con las mejores prácticas del sector y los principios de sostenibilidad tecnológica.



## Dirección del Área de Salud de Tenerife (3 contratos)

- Realización del Servicio Postal Universal del Área de Salud de Tenerife

- La realización de pruebas de Imagen Ósea u Oseométricas y Ortopantomografías a personas migrantes sobre las que existe duda acerca de su mayoría o minoría de edad.

- Servicio de Rehabilitación logopédica



## Director del Instituto Canario de Estadística (ISTAC) (1 contratos)

- Realización en tiempo y con la calidad adecuada, de labores de trabajo de campo del Instituto Canario de Estadística en lassiguientes operaciones estadísticas: a) Índice de Confianza Empresarial b) Estadística de Construcción de Edificios c) Estadísticade Defunciones según la Causa de Muerte



## Fundación Canaria Instituto de Investigación Sanitaria de Canarias (FIISC) (1 contratos)

- Contratación servicio de desarrollo e implementación de la interoperabilidad de FUNDANET-CTMS



## Gerencia de Gestión de Servicios para la Salud y Seguridad en Canarias, S.A. (1 contratos)

- Prestación de los servicios de soporte y desarrollo evolutivo de la Plataforma Business Central On Line



## Gerencia de Servicio Sanitarios de Fuerteventura (1 contratos)

- Servicio de gestión integral y de eliminación de residuos sanitarios y otros residuos, en la Gerencia de Servicios Sanitarios del Área de Salud de Fuerteventura



## Gerencia de Servicios Sanitarios de La Gomera (2 contratos)

- Servicio de mantenimiento integral de la resonancia magnética nuclear marca Philips y el sistema de postproceso de los estudios, intellispace portal solutions, ubicado en el Servicio Radiología del Hospital Nuestra Señora de Guadalupe

- Servicio de mantenimiento de los sistemas de Alimentación Ininterrumpida General Electric Invertomatic, instalados en el Hospital Nuestra Señora de Guadalupe.



## Gerencia de Servicios Sanitarios del Área de Salud de Lanzarote (8 contratos)

- Expediente administrativo para la contratación del servicio de  mantenimiento, en régimen de colaboración técnica, de varios equipos de diagnóstico por imagen de la casa Philips Ibérica, S.A., para el Hospital Universitario Doctor José Molina Orosa, depediente de la Gerencia de Servicios Sanitarios del Área de Salud de Lanzarote.

- El objeto del contrato será la realización del Servicio de sistema integral de reclutamiento y selección de personal técnico, conel fin de dotar las necesidades del Servicio de Recursos Humanos, para cubrir puestos específicos de alta cualificación.

- La realización del Servicio de recogida, carga, transporte, descarga y entrega de mercancías, entre la Nave Central sita en la calle Océano Glacial Ártico nº 33 nave 59B y los Centros dependientes de la GSSLZ del Área de Salud de Lanzarote, con el fin de garantizar la distribución de la mercancía asistencial entre los distintos centros dependientes de la misma en beneficio de los usuarios 

- Expediente administrativo para la contratación del servicio de  mantenimiento, en régimen de colaboración técnica, de varios equipos de diagnóstico por imagen de la casa Philips Ibérica, S.A., para el Hospital Universitario Doctor José Molina Orosa, depediente de la Gerencia de Servicios Sanitarios del Área de Salud de Lanzarote.

- Servicio de gestión integral y de eliminación de residuos sanitarios derivado de acuerdo marco

- Expediente administrativo para la contratación del servicio de  mantenimiento, en régimen de colaboración técnica, de varios equipos de diagnóstico por imagen de la casa Philips Ibérica, S.A., para el Hospital Universitario Doctor José Molina Orosa, depediente de la Gerencia de Servicios Sanitarios del Área de Salud de Lanzarote.

- Servicio de realización de determinadas pruebas analíticas por laboratorio externo para el Hospital Universitario Dr. Jose Molina Orosa.


- Servicio de mantenimiento en régimen de colaboración técnica de dos incubadoras y tres cunas con respiración de soporte vital, dela casa General Electric Healthcare España S.A.U.



## Gerencia de la Fundación Canaria de Juventud Ideo (4 contratos)

- Prestación del servicio de actividades educativas para la Fundación Canaria de Juventud IDEO, mediante procedimiento abierto simplificado y tramitación ordinaria.

- Prestación del servicio de auditoría anual de cuentas y asesoramiento fiscal para la Fundación Canaria de Juventud IDEO

- Prestación del servicio de auditoría anual de cuentas y asesoramiento fiscal para la Fundación Canaria de Juventud IDEO

- Prestación del servicio de actividades educativas para la Fundación Canaria de Juventud IDEO, mediante procedimiento abierto simplificado y tramitación ordinaria.



## Gerencia de la Fundación Tutelar Canaria para la Acción Social, M.P. (FuCAS)  (1 contratos)

-  El objeto del contrato será la realización de los servicios complementarios y auxiliares del CPEE La Casita de la Fundación Tutelar Canaria para La Acción Social, M.P.



## Gerencia de los servicios Sanitarios del Area de Salud de La Palma (1 contratos)

- Servicio de elaboración o actualización y, posterior implantación del programa de análisis de peligros y puntos de controlcríticos (APPCC), en las instalaciones de la cocina del Hospital Universitario de La Palma, centro dependiente de la Gerencia delos Servicios Sanitarios del Área de Salud de La Palma.



## Gestión de Medio Rural de Canarias S.A.U. (GMR Canarias) (13 contratos)

- Servicio de seguros a adjudicar por Gestión del Medio Rural de Canarias, S.A.U.

- Servicio de seguridad, suministro de equipos en régimen de alquiler, servicio de mantenimiento de alarmas anti-intrusión, conexión a central receptora de alarmas, servicio de respuesta ante situaciones de alarma del objeto social de GMR Canarias.

- Servicio de mensajería, distribución de paquetería, envío de burofax para Gestión del Medio Rural de Canarias S.A.U

- Servicio de control in situ a los operadores beneficiarios de la ayuda mediante el REA

- Servicio de lavandería para la uniformidad de trabajo del personal técnico del Área de Ganadería de GMR Canarias, consistente en la recogida, lavado, desinfección y entrega de los uniformes en condiciones óptimas de higiene y conservación. 

- Servicio de realización de diferentes analíticas en el ámbito de la seguridad alimentaria.

- Servicio de seguridad, suministro de equipos en régimen de alquiler, servicio de mantenimiento de alarmas anti-intrusión, conexión a central receptora de alarmas, servicio de respuesta ante situaciones de alarma del objeto social de GMR Canarias.

- Servicio de mensajería, distribución de paquetería, envío de burofax para Gestión del Medio Rural de Canarias S.A.U

- El servicio de seguro de crédito a clientes, con el fin de proteger a Gestión del Medio Rural de Canarias S.A.U. (en adelante, GMR Canarias), frente al riesgo de impago por parte de sus clientes en la comercialización y distribución de productos canarios.

- Servicio de Telecomunicaciones para Gestión del Medio Rural de Canarias, S.A.U

- Servicio de implantación de Dynamics 365 Business Central en la nube para la realización del objeto social de G.M.R. Canarias S.A.U.

- Servicio de realización de diferentes analíticas en el ámbito de la seguridad alimentaria.

- Servicio de seguros a adjudicar por Gestión del Medio Rural de Canarias, S.A.U.



## Presidencia del Ente Público Empresarial Puertos Canarios (3 contratos)

- Servicio de mantenimiento de las instalaciones eléctricas de los puertos de la provincia de Santa Cruz de Tenerife.

- Servicio de comunicación integral destinado al Ente Público Empresarial Puertos Canarios adscrito a la consejería de Obras Públicas, Vivienda y Movilidad del Gobierno de Canarias.

- Consultoría para la definición y descripción de puestos de trabajo en el ente Público Empresarial Puertos Canarios.



## Presidencia del Ente Público Radiotelevisión Canaria (1 contratos)

- Servicio de seguimiento de medios de comunicación para el grupo audiovisual RTVC



## Promotur Turismo Canarias S.A. (3 contratos)

- Patrocinio del evento Insularia islas del mundo en red

- Servicio de asistencia y gestión integral de registro de jornada laboral y suministro de los dispositivos electrónicos de control

- Patrocinio del evento Concierto Il Divo Gira XX Aniversario



## Radio Pública de Canarias, S.A. (2 contratos)

- Aumento de capacidad y servicio de mantenimiento y soporte del sistema de gestión de audio y noticias para Radio Pública de Canarias S.A

- Aumento de capacidad y servicio de mantenimiento y soporte del sistema de gestión de audio y noticias para Radio Pública de Canarias S.A



## Secretaria General de la Presidencia del Gobierno de Canarias (1 contratos)

- El objeto del contrato será la realización del servicio de limpieza e higienización de la sede de la Presidencia del Gobierno en Las Palmas de Gran Canaria. 




## Secretaria General del Servicio Canario de Salud (1 contratos)

- Servicio de consultoría para el desarrollo y actualización del Plan Estratégico de Transformación e Innovación en Salud Digital del Servicio Canario de la Salud



## Secretaría General Técnica de la Consejería de Agricultura, Ganadería, Pesca y Soberanía Alimentaria del Gobierno de Canarias (6 contratos)

- El objeto del contrato será la prestación de servicios de desarrollo y mantenimiento que permitan la creación de una aplicación informática para la gestión de expedientes administrativos que permita al Servicio de Estructuras Pesqueras, mantener un registro y control eficiente de todos los trámites, fases y documentos relacionados con sus procesos administrativos 

- Servicio consistente en el mantenimiento integral de diversos sistemas de información de la Consejería de Agricultura, Ganadería, Pesca y Soberanía Alimentaria

- Servicio consistente en el mantenimiento integral de diversos sistemas de información de la Consejería de Agricultura, Ganadería, Pesca y Soberanía Alimentaria

- Servicio consistente en el mantenimiento integral de diversos sistemas de información de la Consejería de Agricultura, Ganadería, Pesca y Soberanía Alimentaria

- Limpieza de las instalaciones de la Consejería de Agricultura, Ganadería, Pesca y Soberanía Alimentaría, del Servicio de Sanidad Animal y Laboratorio de Calidad Agroalimentaria sitos en San Cristóbal del La Laguna.

- Servicio consistente en el mantenimiento integral de diversos sistemas de información de la Consejería de Agricultura, Ganadería, Pesca y Soberanía Alimentaria



## Secretaría General Técnica de la Consejería de Bienestar Social, Igualdad, Juventud, Infancia y Familias (1 contratos)

- El objeto del contrato será la realización del servicio de análisis y elaboración de formularios electrónicos en webform, en elarco de las instrucciones que al respecto fija la Dirección General de Transformación Digital de los Servicios Públicos



## Secretaría General Técnica de la Consejería de Economía, Industria, Comercio y Autónomos (1 contratos)

- La contratación del servicio consistente en el soporte y mantenimiento de la aplicación informática denominada Sistema Integradode Información Económica (SIIE2)



## Servicio Canario de Empleo (2 contratos)

- Servicio para la gestión de las ferias y congreso de empleo 2025, para el Servicio Canario de Empleo

- Servicio para la gestión de las ferias y congreso de empleo 2025, para el Servicio Canario de Empleo



## Viceconsejería de Cultura y Patrimonio Cultural del Gobierno de Canarias (1 contratos)

- Servicio de limpieza de centros de la Dirección General de Cultura y Patrimonio Cultural en la provincia de Santa Cruz de Tenerife: dependencias de Comodoro Rolín, Sala de Arte Contemporáneo en Comodoro Rolín, Espacio Cultural El Tanque en C/Adán Martín y IES Cabrera Pinto (Sala de exposiciones) en San Cristóbal de La Laguna



## Viceconsejería de la Presidencia del Gobierno de Canarias (1 contratos)

- Cesión de derechos de imagen del deportista de élite Pedro González López "Pedri" para la promoción de la campaña "Talento Canario"


